In [36]:
import os
import re
import sqlite3
import json
from operator import itemgetter

import numpy as np
import pandas as pd
import matplotlib as plt

from Bio import SeqIO

from IPython.display import display

In [26]:
### Copy-pasted from another notebook ###

import os
import json
import random

import numpy as np
import pandas as pd

from IPython.display import display

N_SPECIAL_TOKENS = 2
PAD_TOKEN = 0
MASK_TOKEN = 1

PROT_LETTERS = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

MAX_SEQ_LEN = 1000
N_ANNOTATIONS = 5000

annotation_counts = pd.read_csv('/cs/phd/nadavb/cafa_project/data/unique_annotations_counts.csv', \
        index_col = 0, squeeze = True, names = ['count'])
assert len(annotation_counts) >= N_ANNOTATIONS
used_annotation_counts = annotation_counts.iloc[:N_ANNOTATIONS]
print('The rarest used annotation appears %d times.' % used_annotation_counts.min())
unique_annotations = list(used_annotation_counts.index)
aa_to_token = {aa: i + N_SPECIAL_TOKENS for i, aa in enumerate(PROT_LETTERS)}
annotation_to_index = {annotation: i for i, annotation in enumerate(unique_annotations)}
unique_annotations_set = set(unique_annotations)

def encode_seqs(batch_seqs, max_len = MAX_SEQ_LEN):
    
    encoded_seqs = np.full((len(batch_seqs), max_len), PAD_TOKEN, dtype = np.int8)
    
    for i, seq in enumerate(batch_seqs):
        assert len(seq) <= max_len
        encoded_seqs[i, :len(seq)] = [aa_to_token[aa] for aa in seq]
    
    return encoded_seqs

def encode_annotations(batch_annotations):
    
    encoded_annotations = np.zeros((len(batch_annotations), len(unique_annotations)), dtype = np.int8)
    
    for i, annotations in enumerate(encoded_annotations):
        for annotation in annotations:
            if annotation in annotation_to_index:
                encoded_annotations[i, annotation_to_index[annotation]] = 1
                
    return encoded_annotations
                
#########################################

The rarest used annotation appears 477 times.


In [16]:
go_annotations_meta = pd.read_csv('/cs/labs/michall/nadavb/cafa_project/data/go_annotations.csv', index_col = 'index')
go_annotation_index_to_id = go_annotations_meta['id']
display(go_annotations_meta)

,id,name,namespace,def,is_a,synonym,alt_id,subset,is_obsolete,xref,...,replaced_by,created_by,creation_date,property_value,direct_children,direct_parents,all_ancestors,all_offsprings,count,freq
index,,,,,,,,,,,,,,,,,,,,,
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","['GO:0048308 ! organelle inheritance', 'GO:004...","['""mitochondrial inheritance"" EXACT []']",[],[],False,[],...,NaN,NaN,NaN,[],set(),"{'GO:0048308', 'GO:0048311'}","{'GO:0051640', 'GO:0071840', 'GO:0051179', 'GO...",{'GO:0000001'},0.0,0.000000e+00
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",['GO:0007005 ! mitochondrion organization'],[],[],[],False,[],...,NaN,NaN,NaN,[],{'GO:0033955'},{'GO:0007005'},"{'GO:0071840', 'GO:0000002', 'GO:0009987', 'GO...",{'GO:0000002'},4301.0,9.254727e-05
2,GO:0000003,reproduction,biological_process,"""The production of new individuals that contai...",['GO:0008150 ! biological_process'],"['""reproductive physiological process"" EXACT []']","['GO:0019952', 'GO:0050876']","['goslim_agr', 'goslim_chembl', 'goslim_flybas...",False,['Wikipedia:Reproduction'],...,NaN,NaN,NaN,[],"{'GO:0032504', 'GO:0032505', 'GO:0061887', 'GO...",{'GO:0008150'},{'GO:0000003'},"{'GO:0030436', 'GO:0007114', 'GO:0043936', 'GO...",181.0,3.894689e-06
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,"""OBSOLETE. Assists in the correct assembly of ...",[],"['""ribosomal chaperone activity"" EXACT []']",[],[],True,[],...,NaN,NaN,NaN,[],set(),set(),{'GO:0000005'},{'GO:0000005'},0.0,0.000000e+00
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...",['GO:0005385 ! zinc ion transmembrane transpor...,"['""high affinity zinc uptake transmembrane tra...",[],[],False,[],...,NaN,NaN,NaN,[],set(),{'GO:0005385'},"{'GO:0046915', 'GO:0005385', 'GO:0022857', 'GO...",{'GO:0000006'},10.0,2.151762e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47370,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways involving...","['GO:0006040 ! amino sugar metabolic process',...","['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...",[],[],False,[],...,NaN,pr,2012-03-22T01:19:54Z,[],"{'GO:2001315', 'GO:2001314'}","{'GO:0006040', 'GO:0006793', 'GO:0009225'}","{'GO:0034641', 'GO:0006040', 'GO:0044238', 'GO...",{'GO:2001313'},0.0,0.000000e+00
47371,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...",['GO:0009227 ! nucleotide-sugar catabolic proc...,"['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...",[],[],False,[],...,NaN,pr,2012-03-22T01:20:01Z,[],set(),"{'GO:0009227', 'GO:0046348', 'GO:2001313'}","{'GO:0034641', 'GO:0046348', 'GO:0009227', 'GO...",{'GO:2001314'},0.0,0.000000e+00
47372,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...",['GO:0009226 ! nucleotide-sugar biosynthetic p...,"['""UDP-4-deoxy-4-formamido-beta-L-arabinopyran...",[],[],False,[],...,NaN,pr,2012-03-22T01:20:05Z,[],set(),"{'GO:2001313', 'GO:0009226', 'GO:0046349'}","{'GO:0034641', 'GO:0019438', 'GO:0046349', 'GO...",{'GO:2001315'},0.0,0.000000e+00


In [3]:
# wget https://www.biofunctionprediction.org/cafa-targets/CAFA4-export.tgz
# tar -xzvf CAFA4-export.tgz
TARGET_FILES_DIR = '/cs/phd/nadavb/cafa_project/data/CAFA4-export/TargetFiles'

# From 00README.txt
TAXA_IDS = {
    9606,    # "Homo sapiens"
    10090,   # "Mus musculus[All Names]"
    10116,   # "Rattus norvegicus"
    3702,    # "Arabidopsis thaliana[All Names]"
    83333,   # "Escherichia coli K-12[all names]" P
    7227,    # "Drosophila melanogaster[All Names]"
    287,     # "Pseudomonas aeruginosa[All Names]" P
    559292,  # "Saccharomyces cerevisiae ATCC 204508"
    284812,  # "Schizosaccharomyces pombe ATCC 24843"
    7955,    # "Danio rerio[All Names]"
    44689,   # "Dictyostelium discoideum[All Names]"
    243273,  # "Mycoplasma genitalium ATCC 33530" P
    6239,    # "Caenorhabditis elegans[All Names]"
    226900,  # "Bacillus cereus ATCC 14579" P
    4577,    # "Zea Mays [All names]"
    9823,    # "Sus scrofa"
    99287,   # Salmonella typhymurium ATCC 700720
}

TARGET_FILE_PATTERN = re.compile(r'sp_species\.(\d+)\.tfa')
TARGET_SEQ_DESCRIPTION_PATTERN = re.compile(r'(\S+) (\S+)')

parsed_taxa_ids = {int(TARGET_FILE_PATTERN.match(f_name).group(1)) for f_name in os.listdir(TARGET_FILES_DIR)}
assert parsed_taxa_ids == TAXA_IDS

target_seqs = []

for taxa_id in TAXA_IDS:
    for record in SeqIO.parse(os.path.join(TARGET_FILES_DIR, 'sp_species.%d.tfa' % taxa_id), 'fasta'):
        target_seqs.append((taxa_id,) + TARGET_SEQ_DESCRIPTION_PATTERN.match(record.description).groups() + \
                (str(record.seq),))

target_seqs = pd.DataFrame(target_seqs, columns = ['taxa_id', 'cafa_id', 'uniprot_name', 'seq'])
display(target_seqs)

,taxa_id,cafa_id,uniprot_name,seq
0,4577,T45770000001,14331_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...
1,4577,T45770000002,14332_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...
2,4577,T45770000003,1433X_MAIZE,ILNSPDRACNLAKQAFDEAISELDSLGEESYKDSTLIMQLLXDNLT...
3,4577,T45770000004,708A6_MAIZE,MAANGGDHTSARPHVVLLPSAGMGHLVPFARLAVALSEGHGCNVSV...
4,4577,T45770000005,AAM1I_MAIZE,MPMRIERDLHMAIGNGETSYTKNSRIQEKAMFQMKSVLEEATRAVC...
...,...,...,...,...
97994,287,T2870000073,TRA6_PSEAI,MINVAILSAIRRWHFRDGASIREIARRSGLSRNTVRKYLQSKVVEP...
97995,287,T2870000074,WBJE_PSEAI,MARIFVVSEYVGANQNSTGYYWEKIIGKMQREFGGLTVIFPLTAGE...
97996,287,T2870000075,XER_PSEAI,MTPQQLTEEYIFAHDLREASAKIYRAATKALLKHFGPTATVQDVDH...
97997,287,T2870000076,XYLE_PSEAI,MNKGIMRPGHVQLRVLDMSKALEHYVELLGLIEMDRDDQGRVYLKA...


In [8]:
ANNOTATIONS_SQLITE_FILE_PATH = '/cs/phd/nadavb/cafa_project/data/protein_annotations.db'

sql_uniprot_names = '(' + ','.join(target_seqs['uniprot_name'].apply(lambda name: '\'%s\'' % name).tolist()) + ')'

cnx = sqlite3.connect(ANNOTATIONS_SQLITE_FILE_PATH)
go_annots = pd.read_sql_query('SELECT * FROM protein_annotations WHERE uniprot_name IN %s' % sql_uniprot_names, cnx)
display(go_annots)

,index,tax_id,uniprot_name,go_annotations,flat_go_annotations,n_go_annotations,complete_go_annotation_indices,n_complete_go_annotations
0,47,10090.0,TITIN_MOUSE,"{""GO Molecular Function"": [], ""GO Biological P...",[],0,[],0
1,110,9606.0,TITIN_HUMAN,"{""GO Molecular Function"": [], ""GO Biological P...",[],0,[],0
2,1055,6239.0,TTN1_CAEEL,"{""GO Molecular Function"": [""GO:0004672"", ""GO:0...","[""GO:0004672"", ""GO:0005524""]",2,"[3561, 4205]",2
3,1098,7227.0,TITIN_DROME,"{""GO Molecular Function"": [], ""GO Biological P...",[],0,[],0
4,1795,9606.0,MUC16_HUMAN,"{""GO Molecular Function"": [], ""GO Biological P...","[""GO:0002223"", ""GO:0005796"", ""GO:0005886"", ""GO...",9,"[1545, 4451, 4526, 5664, 10019, 10229, 12748, ...",9
...,...,...,...,...,...,...,...,...
83663,106551905,9606.0,MORN_HUMAN,"{""GO Molecular Function"": [], ""GO Biological P...",[],0,[],0
83664,106551944,7227.0,TAL1A_DROME,"{""GO Molecular Function"": [], ""GO Biological P...","[""GO:0002009"", ""GO:0005634"", ""GO:0005737"", ""GO...",5,"[1339, 4304, 4396, 5543, 5775]",5
83665,106551945,7227.0,TAL3A_DROME,"{""GO Molecular Function"": [], ""GO Biological P...",[],0,[],0
83666,106551967,10090.0,UF05_MOUSE,"{""GO Molecular Function"": [], ""GO Biological P...",[],0,[],0


In [12]:
uniprot_name_to_complete_go_annotation_indices = go_annots.set_index('uniprot_name')['complete_go_annotation_indices']
target_seqs['complete_go_annotation_indices'] = target_seqs['uniprot_name'].map(\
        uniprot_name_to_complete_go_annotation_indices)
display(target_seqs)

,taxa_id,cafa_id,uniprot_name,seq,complete_go_annotation_indices
0,4577,T45770000001,14331_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,NaN
1,4577,T45770000002,14332_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,"[4304, 8388, 12754, 33482]"
2,4577,T45770000003,1433X_MAIZE,ILNSPDRACNLAKQAFDEAISELDSLGEESYKDSTLIMQLLXDNLT...,[12754]
3,4577,T45770000004,708A6_MAIZE,MAANGGDHTSARPHVVLLPSAGMGHLVPFARLAVALSEGHGCNVSV...,[10628]
4,4577,T45770000005,AAM1I_MAIZE,MPMRIERDLHMAIGNGETSYTKNSRIQEKAMFQMKSVLEEATRAVC...,NaN
...,...,...,...,...,...
97994,287,T2870000073,TRA6_PSEAI,MINVAILSAIRRWHFRDGASIREIARRSGLSRNTVRKYLQSKVVEP...,[]
97995,287,T2870000074,WBJE_PSEAI,MARIFVVSEYVGANQNSTGYYWEKIIGKMQREFGGLTVIFPLTAGE...,[]
97996,287,T2870000075,XER_PSEAI,MTPQQLTEEYIFAHDLREASAKIYRAATKALLKHFGPTATVQDVDH...,[]
97997,287,T2870000076,XYLE_PSEAI,MNKGIMRPGHVQLRVLDMSKALEHYVELLGLIEMDRDDQGRVYLKA...,"[12321, 27822]"


In [22]:
target_seqs.to_csv('/cs/phd/nadavb/cafa_project/data/target_seqs.csv')

# Check Dan's data

In [40]:
target_seqs = pd.read_csv('/cs/phd/nadavb/cafa_project/data/target_seqs.csv', index_col = 0)
display(target_seqs)

target_seqs_ext = pd.read_csv('/cs/phd/nadavb/cafa_project/data/target_seqs_expanded_annotations.csv.gz')
display(target_seqs_ext)

,taxa_id,cafa_id,uniprot_name,seq,complete_go_annotation_indices
0,4577,T45770000001,14331_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,NaN
1,4577,T45770000002,14332_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,"[4304, 8388, 12754, 33482]"
2,4577,T45770000003,1433X_MAIZE,ILNSPDRACNLAKQAFDEAISELDSLGEESYKDSTLIMQLLXDNLT...,[12754]
3,4577,T45770000004,708A6_MAIZE,MAANGGDHTSARPHVVLLPSAGMGHLVPFARLAVALSEGHGCNVSV...,[10628]
4,4577,T45770000005,AAM1I_MAIZE,MPMRIERDLHMAIGNGETSYTKNSRIQEKAMFQMKSVLEEATRAVC...,NaN
...,...,...,...,...,...
97994,287,T2870000073,TRA6_PSEAI,MINVAILSAIRRWHFRDGASIREIARRSGLSRNTVRKYLQSKVVEP...,[]
97995,287,T2870000074,WBJE_PSEAI,MARIFVVSEYVGANQNSTGYYWEKIIGKMQREFGGLTVIFPLTAGE...,[]
97996,287,T2870000075,XER_PSEAI,MTPQQLTEEYIFAHDLREASAKIYRAATKALLKHFGPTATVQDVDH...,[]
97997,287,T2870000076,XYLE_PSEAI,MNKGIMRPGHVQLRVLDMSKALEHYVELLGLIEMDRDDQGRVYLKA...,"[12321, 27822]"


,taxa_id,cafa_id,uniprot_name,seq,complete_go_annotation_indices
0,4577,T45770000001,14331_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,[]
1,4577,T45770000002,14332_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,"[1265, 1266, 4178, 4196, 4304, 5053, 7832, 838..."
2,4577,T45770000003,1433X_MAIZE,ILNSPDRACNLAKQAFDEAISELDSLGEESYKDSTLIMQLLXDNLT...,"[4178, 4196, 12754]"
3,4577,T45770000004,708A6_MAIZE,MAANGGDHTSARPHVVLLPSAGMGHLVPFARLAVALSEGHGCNVSV...,"[2818, 10611, 10627, 10628]"
4,4577,T45770000005,AAM1I_MAIZE,MPMRIERDLHMAIGNGETSYTKNSRIQEKAMFQMKSVLEEATRAVC...,[]
...,...,...,...,...,...
97994,287,T2870000073,TRA6_PSEAI,MINVAILSAIRRWHFRDGASIREIARRSGLSRNTVRKYLQSKVVEP...,[]
97995,287,T2870000074,WBJE_PSEAI,MARIFVVSEYVGANQNSTGYYWEKIIGKMQREFGGLTVIFPLTAGE...,[]
97996,287,T2870000075,XER_PSEAI,MTPQQLTEEYIFAHDLREASAKIYRAATKALLKHFGPTATVQDVDH...,[]
97997,287,T2870000076,XYLE_PSEAI,MNKGIMRPGHVQLRVLDMSKALEHYVELLGLIEMDRDDQGRVYLKA...,"[2818, 5298, 6238, 7043, 7924, 10387, 12321, 2..."


In [42]:
(target_seqs == target_seqs_ext).all()

taxa_id                            True
cafa_id                            True
uniprot_name                       True
seq                                True
complete_go_annotation_indices    False
dtype: bool

In [44]:
def parse_annotations_as_set(raw_annotations):
    if pd.isnull(raw_annotations) or raw_annotations == '[]':
        return set()
    else:
        return set(json.loads(raw_annotations))
    
original_annotations = target_seqs['complete_go_annotation_indices'].apply(parse_annotations_as_set)
ext_annotations = target_seqs_ext['complete_go_annotation_indices'].apply(parse_annotations_as_set)
all(org_annot <= ext_annot for org_annot, ext_annot in zip(original_annotations, ext_annotations))

True

# Final code

In [45]:
target_seqs = pd.read_csv('/cs/phd/nadavb/cafa_project/data/target_seqs_expanded_annotations.csv.gz')
display(target_seqs)

,taxa_id,cafa_id,uniprot_name,seq,complete_go_annotation_indices
0,4577,T45770000001,14331_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,[]
1,4577,T45770000002,14332_MAIZE,MASAELSREENVYMAKLAEQAERYEEMVEFMEKVAKTVDSEELTVE...,"[1265, 1266, 4178, 4196, 4304, 5053, 7832, 838..."
2,4577,T45770000003,1433X_MAIZE,ILNSPDRACNLAKQAFDEAISELDSLGEESYKDSTLIMQLLXDNLT...,"[4178, 4196, 12754]"
3,4577,T45770000004,708A6_MAIZE,MAANGGDHTSARPHVVLLPSAGMGHLVPFARLAVALSEGHGCNVSV...,"[2818, 10611, 10627, 10628]"
4,4577,T45770000005,AAM1I_MAIZE,MPMRIERDLHMAIGNGETSYTKNSRIQEKAMFQMKSVLEEATRAVC...,[]
...,...,...,...,...,...
97994,287,T2870000073,TRA6_PSEAI,MINVAILSAIRRWHFRDGASIREIARRSGLSRNTVRKYLQSKVVEP...,[]
97995,287,T2870000074,WBJE_PSEAI,MARIFVVSEYVGANQNSTGYYWEKIIGKMQREFGGLTVIFPLTAGE...,[]
97996,287,T2870000075,XER_PSEAI,MTPQQLTEEYIFAHDLREASAKIYRAATKALLKHFGPTATVQDVDH...,[]
97997,287,T2870000076,XYLE_PSEAI,MNKGIMRPGHVQLRVLDMSKALEHYVELLGLIEMDRDDQGRVYLKA...,"[2818, 5298, 6238, 7043, 7924, 10387, 12321, 2..."


In [37]:
def process_submission_batch(prediction_model, formatted_predictions, batch_cafa_ids, batch_seqs, \
        batch_go_annotation_indices):
    
    MAX_ANNOTATIONS_PER_PROTEIN = 1500
    
    batch_annotation_predictions = prediction_model(batch_seqs, batch_go_annotation_indices)
    
    for cafa_id, annotation_predictions in zip(batch_cafa_ids, batch_annotation_predictions):
        top_predicted_annotations = list(sorted(annotation_predictions.items(), key = itemgetter(1), reverse = True))\
                [:MAX_ANNOTATIONS_PER_PROTEIN]
        top_predicted_annotations = [(annotation_index, round(score, 2)) for annotation_index, score in \
                top_predicted_annotations]
        formatted_predictions.extend(['%s\t%s\t%.2f' % (cafa_id, go_annotation_index_to_id[annotation_index], score) for \
                annotation_index, score in top_predicted_annotations if score > 0])

def create_submission_files(prediction_model, batch_size, output_dir, model_id = 1):
    
    '''
    prediction model is expected to be a function that takes two arguments (seqs and annotations) and returns the final,
    refined annotations. The function is expected to work in batch (i.e. receive multiple inputs and produce multiple 
    outputs). The protein seqs is expected as a list of strings of aa letters. The input annotations are expected
    as a list of list of integers, as provided in the SQLITE DB. There should be full correspondence between each seq
    string to each set of annotations (list of integers); each pair is considered a distinct protein. The output
    annotations is expected as a list of dictionaries, mapping each annotation integer (the same indices provided by
    the SQLITE DB) into a confidence score between 0 to 1. Each output dictionary corresponds to the corresponding input
    protein.
    
    See: https://www.biofunctionprediction.org/cafa-targets/CAFA4_rules%20_01_2020_v4.pdf
    '''
    
    OUTPUT_FILE_NAME_PATTERN = 'linialgroup_%d_%s_go.txt'
    SUBMISSION_FILE_PREFIX = 'AUTHOR Linial' + '\n' + ('MODEL %d' % model_id) + '\n' + 'KEYWORDS machine learning.'
    
    for tax_id, tax_target_seqs in target_seqs.groupby('taxa_id'):
        
        print('Preparing submission for tax ID %s...' % tax_id)
        formatted_predictions = []
        batch_cafa_ids = []
        batch_seqs = []
        batch_go_annotation_indices = []
        
        for _, (cafa_id, seq, raw_go_annotation_indices) in tax_target_seqs[['cafa_id', 'seq', \
                'complete_go_annotation_indices']].iterrows():
            
            batch_cafa_ids.append(cafa_id)
            batch_seqs.append(seq)
            batch_go_annotation_indices.append([] if pd.isnull(raw_go_annotation_indices) else \
                    json.loads(raw_go_annotation_indices))
            
            if len(batch_cafa_ids) >= batch_size:
                process_submission_batch(prediction_model, formatted_predictions, batch_cafa_ids, batch_seqs, \
                        batch_go_annotation_indices)
                batch_cafa_ids = []
                batch_seqs = []
                batch_go_annotation_indices = []
                
        if len(batch_cafa_ids) > 0:
            process_submission_batch(prediction_model, formatted_predictions, batch_cafa_ids, batch_seqs, \
                    batch_go_annotation_indices)
            
        tax_submission_content = SUBMISSION_FILE_PREFIX + '\n' + '\n'.join(formatted_predictions) + '\n' + 'END'
        
        with open(os.path.join(output_dir, OUTPUT_FILE_NAME_PATTERN % (model_id, tax_id)), 'w') as f:
            f.write(tax_submission_content)
            
    print('Done.')
    
def dummy_prediction_model(batch_seqs, batch_go_annotation_indices):
    
    batch_refined_go_annotation_indices = []
    
    for go_annotation_indices in batch_go_annotation_indices:
        
        refined_go_annotation_indices = {go_annotation_index: 1.0 for go_annotation_index in go_annotation_indices}
    
        for go_annotation_index in np.random.choice(go_annotations_meta.index, np.random.randint(100), replace = False):
            refined_go_annotation_indices[go_annotation_index] = np.random.rand()

        batch_refined_go_annotation_indices.append(refined_go_annotation_indices)
        
    return batch_refined_go_annotation_indices

create_submission_files(dummy_prediction_model, 128, '/cs/phd/nadavb/tmp/dummy_cafa_submission/')

Preparing submission for tax ID 287...
Preparing submission for tax ID 3702...


KeyboardInterrupt: 

In [38]:
def create_submission_files_with_keras_model(keras_model, batch_size, output_dir, model_id = 1):
    
    def prediction_model(batch_seqs, batch_annotations):

        max_len = max(map(len, batch_seqs))
        batch_encoded_seqs = encode_seqs(batch_seqs, max_len = max_len)
        batch_encoded_annotations = encode_annotations(batch_annotations)
                
        _, batch_pred_annotation_scores = keras_model.predict(batch_encoded_seqs, batch_encoded_annotations)
        batch_annotation_predictions_as_dicts = []
        
        for (annotations, pred_annotation_scores) in zip(batch_annotations, batch_pred_annotation_scores):
            annotation_predictions_as_dicts = {unique_annotations[i]: score for i, score in \
                    enumerate(pred_annotation_scores)}
            annotation_predictions_as_dicts.update({annotation: 1.0 for annotation in annotations})
            batch_annotation_predictions_as_dicts.append(annotation_predictions_as_dicts)
            
        return batch_annotation_predictions_as_dicts
    
    create_submission_files(prediction_model, batch_size = batch_size, output_dir = output_dir, model_id = model_id) 
        
class DummyKerasModel:
    def predict(self, batch_encoded_seqs, batch_encoded_annotations):
        return None, np.random.rand(*batch_encoded_annotations.shape)
    
create_submission_files_with_keras_model(DummyKerasModel(), 128, '/cs/phd/nadavb/tmp/dummy_cafa_submission/')

Preparing submission for tax ID 287...
Preparing submission for tax ID 3702...
Preparing submission for tax ID 4577...
Preparing submission for tax ID 6239...
Preparing submission for tax ID 7227...
Preparing submission for tax ID 7955...
Preparing submission for tax ID 9606...
Preparing submission for tax ID 9823...
Preparing submission for tax ID 10090...


KeyboardInterrupt: 